In [1]:
import pandas as pd
import numpy as np
def get_group_columns(df: pd.DataFrame, group: str):
    """
    get a group of columns from multi-index columns DataFrame

    Parameters
    ----------
    df : pd.DataFrame
        with multi of columns.
    group : str
        the name of the feature group, i.e. the first level value of the group index.
    """
    if group is None:
        return df.columns
    else:
        return df.columns[df.columns.get_loc(group)]

In [2]:
alphabet = np.array(list('abcdefg'))
digits = np.array(list(range(0,10)))

In [3]:
avalue = alphabet[np.random.randint(0,len(alphabet),100)]
dvalue = digits[np.random.randint(0,len(digits),100)]
value = np.random.normal(0, 1, 100)

In [4]:
df = pd.DataFrame({'a':avalue,'d':dvalue,'v':value}).set_index(['a','d'])

In [5]:
get_group_columns(df,'v')

'v'

In [6]:
df

,,v
a,d,
d,6,-0.037356
b,9,-0.367281
g,3,-0.745930
d,3,-1.428008
a,0,0.530183
...,...,...
d,6,-1.331729
c,2,0.276174
d,8,1.770959


In [7]:
mask = np.random.rand(100)<0.1

In [8]:
df['v'][mask] = np.nan

In [9]:
get_group_columns(df, 'v')

'v'

In [10]:
df.loc(axis=0)['e']

,v
d,
1,0.724889
2,0.366817
0,NaN
6,0.146869
2,-1.003600
6,-0.075696
9,0.283460
6,0.285292
5,1.172986


In [57]:
import qlib
from qlib.config import REG_CN
provider_uri = "~/.qlib/qlib_data/cn_data"  # target_dir
qlib.init(provider_uri=provider_uri, region=REG_CN)

[24361:MainThread](2021-05-14 21:11:13,188) INFO - qlib.Initialization - [config.py:275] - default_conf: client.
[24361:MainThread](2021-05-14 21:11:13,195) WARNING - qlib.Initialization - [config.py:291] - redis connection failed(host=127.0.0.1 port=6379), cache will not be used!
[24361:MainThread](2021-05-14 21:11:13,745) INFO - qlib.Initialization - [__init__.py:46] - qlib successfully initialized based on client settings.
[24361:MainThread](2021-05-14 21:11:13,746) INFO - qlib.Initialization - [__init__.py:47] - data_path=/Users/harry/.qlib/qlib_data/cn_data
